# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!


In [48]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [49]:
spark = SparkSession.builder.appName('Exercise').getOrCreate()

In [50]:
data_ship = spark.read.csv('cruise_ship_info.csv', inferSchema = True, header = True)


In [4]:
data_ship.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data_ship.head(2)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [12]:
data_ship.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [15]:
from pyspark.ml.feature import StringIndexer

In [17]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_num')

In [19]:
data_ship_indexed = indexer.fit(data_ship).transform(data_ship)

In [20]:
data_ship_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_num|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [47]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
data_ship_indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_num']

In [73]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_num'], outputCol='features')

In [74]:
output = assembler.transform(data_ship_indexed)
output.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_num=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [75]:
final_data = output.select('features', 'crew')

In [76]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [77]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [78]:
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               117|
|   mean| 7.571794871794881|
| stddev|3.4512030924223938|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                41|
|   mean| 8.428780487804877|
| stddev|3.6163961309029404|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [82]:
lr = LinearRegression(labelCol='crew')

In [83]:
lm_model = lr.fit(train_data)

24/12/26 10:13:12 WARN Instrumentation: [e34d28e4] regParam is zero, which might cause numerical instability and overfitting.


In [84]:
test_result = lm_model.evaluate(test_data)

In [85]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2607791950095226|
|-0.10481090601970866|
|0.023726620638402807|
| -0.9443506594776157|
| -1.0912666134567708|
| 0.45852281708557285|
| -0.3621948500218295|
|-0.43355602804630955|
| -0.1500634361836859|
|   1.186563555359049|
| 0.23337511614865925|
| 0.15959386642347972|
|  1.0322481457848305|
|  0.7734199835939322|
|   7.223225515661241|
| -0.4266905073473257|
|  0.8913137817064616|
|  0.7915701674043856|
| 0.28725051428172144|
|-0.04271826124186129|
+--------------------+
only showing top 20 rows



In [64]:
test_result.meanAbsoluteError

3.741997604310364e-15

In [87]:
test_result.rootMeanSquaredError

1.2731741044904679

In [88]:
test_result.r2

0.8729579576038606

In [89]:
unlabeled_data = test_data.select('features')

In [90]:
predictions = lm_model.transform(unlabeled_data)

In [91]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.260779195009523|
|[5.0,122.0,28.5,1...| 6.804810906019709|
|[5.0,133.5,39.59,...|13.106273379361598|
|[6.0,30.276999999...|4.4943506594776155|
|[6.0,90.0,20.0,9....| 10.09126661345677|
|[6.0,110.23899999...|11.041477182914427|
|[6.0,112.0,38.0,9...| 11.26219485002183|
|[7.0,116.0,31.0,9...| 12.43355602804631|
|[7.0,158.0,43.7,1...|13.750063436183686|
|[9.0,113.0,26.74,...|11.193436444640952|
|[10.0,77.0,20.16,...|  8.76662488385134|
|[11.0,58.6,15.66,...|  7.44040613357652|
|[11.0,90.0,22.4,9...|  9.96775185421517|
|[11.0,91.0,20.32,...| 9.216580016406068|
|[11.0,110.0,29.74...| 11.87677448433876|
|[12.0,58.6,15.66,...| 7.426690507347326|
|[12.0,88.5,21.24,...| 9.398686218293538|
|[12.0,91.0,20.32,...| 9.198429832595615|
|[12.0,108.865,27....|10.712749485718279|
|[13.0,25.0,3.82,5...|2.9927182612418615|
+--------------------+------------

In [92]:
predictions_with_labels = predictions.select('features', 'prediction')

test_results_with_labels = test_data.select('crew', 'features').join(predictions, 'features')

test_results_with_labels.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[5.0,86.0,21.04,9...|  8.0| 9.260779195009523|
|[5.0,122.0,28.5,1...|  6.7| 6.804810906019709|
|[5.0,133.5,39.59,...|13.13|13.106273379361598|
|[6.0,30.276999999...| 3.55|4.4943506594776155|
|[6.0,90.0,20.0,9....|  9.0| 10.09126661345677|
|[6.0,110.23899999...| 11.5|11.041477182914427|
|[6.0,112.0,38.0,9...| 10.9| 11.26219485002183|
|[7.0,116.0,31.0,9...| 12.0| 12.43355602804631|
|[7.0,158.0,43.7,1...| 13.6|13.750063436183686|
|[9.0,113.0,26.74,...|12.38|11.193436444640952|
|[10.0,77.0,20.16,...|  9.0|  8.76662488385134|
|[11.0,58.6,15.66,...|  7.6|  7.44040613357652|
|[11.0,90.0,22.4,9...| 11.0|  9.96775185421517|
|[11.0,91.0,20.32,...| 9.99| 9.216580016406068|
|[11.0,110.0,29.74...| 19.1| 11.87677448433876|
|[12.0,58.6,15.66,...|  7.0| 7.426690507347326|
|[12.0,88.5,21.24,...|10.29| 9.398686218293538|
|[12.0,91.0,20.32,...| 9.99| 9.198429832